## Scipy and Optimization

In the last notebook we took a tour of the `numpy` and `scipy` stack as it relates to math and statistics. This notebook is going to focus on another subset of `scipy` functionality that is absolutely critical to data analytics. Nearly every statistical algorithm aside from Ordinary Least Squares (OLS) regression cannot be solved algebraically. Because they cannot be solved algebraically, they must be solved **numerically**.

### Numeric Optimization

Let's start with an example of optimization. Let's say that you know the demand function for tickets to watch a local sports franchise. You can write the inverse demand function as

$$ P = 300 - \frac{1}{2}Q $$

and the total cost function as

$$ TC = 4000 + 45Q $$

In order to choose the right number of tickets to sell, you need to calculate the quantity of tickets that will maximize profits. We can calculate total revenue as $ TR = P \times Q $, and we can calculate profit as $ \Pi = TR - TC $. This means that our profit function is

$$ \Pi = 300Q - \frac{1}{2}Q^2 - 4000 - 45Q $$

In order to find the $Q$ associated with the highest achievable level of profit, we can use calculus to find the point at which the rate of change in the profit function is zero ($\frac{\partial\Pi}{\partial Q}=0$).

$$ \frac{\partial\Pi}{\partial Q} = 300 - Q - 45 = 0 \implies Q = 255$$

So we can **algebraically** solve this particular problem. This isn't always the case. Using `scipy`, we can solve this same problem, as well as many algebraically intractable problems that might be more interesting to us.


<img src="https://github.com/dustywhite7/Econ8320/raw/master/SlidesCode/paraboloid.png" width="200" height="200" />


In any optimization problem, we need to find a way to get ourselves to the minimum, and to know when we get there. When we look at the above image, we are able to visually trace the functional shape (looks like a rainbow ice cream cone to me...) and locate the bottom of the function. What we want to do is utilize an algorithm to "trace" our way from an arbitrary starting point within a function to the optimal point in that function.

In three or fewer dimensions, this is easy. Regressions and statistical models often live in worlds with 100's or 1000's (even millions sometimes) of dimensions. We can't visualize our way to the bottom of those functions!

The class of algorithm that is used to solve these problems is called **gradient descent**.

<img src="https://github.com/dustywhite7/Econ8320/raw/master/SlidesCode/gradDesc.png" width="400" />

**Gradient descent** is an algorithm that explores the shape of the function, and determines which direction is most likely to lead to the optimal point. Let's focus on minimization. We want to find our way to the *bottom* of a function, and we can use gradient descent to try to get there. Given any starting point, our goal is to check the direction in which we can move downward most quickly, and start moving in that direction. At some distance from our starting point, we will stop and re-evaluate the direction in which we would like to travel. Are we still heading downhill in the steepest direction? If we aren't, then we need to update our behavior.

Our gradient descent algorithm will keep "looking around" and moving down until it reaches a point at which it can no longer move "down" in any meaningful way. That is the stopping point, and is treated as the optimum.


With an intuitive understanding of how optimization will happen computationally, it's time to learn a bit more about the math and the code that will help us to achieve computational optimization.

Consider a function, $f$, with two variables $x$ and $y$. Because there are two input variables in the function, it has two partial derivatives:

$$ \frac{\partial f}{\partial x} \text{ and } \frac{\partial f}{\partial y} $$

Each partial derivative tells us how $f$ changes as we move in a particular dimension **all else equal**. The **gradient**, then, is the vector of all partial derivatives of a given function at any point along the function:

$$ \nabla f = \left[ \begin{matrix} \frac{\partial f}{\partial x} \\ \\ \frac{\partial f}{\partial y} \end{matrix} \right]  $$

We can use the gradient to determine the linear approximation of a function at any given point. Think about the gradient as the mathematical representation of the slope and direction of a hill you are hiking on. If we know the gradient, we know which way is down. As we continue to calculate gradients while walking, we can continue to ensure that we will walk downhill until we reach the bottom of the hill.




The steps of our gradient descent function will be the following:

- Evaluate the gradient of the function
- Find the direction of steepest descent
- Determine how far to move in that direction
- Move to new point
- Reevaluate the gradient
- Stop moving when gradient is within a margin of error from 0

Let's try to implement gradient descent by solving our old profit maximization problem computationally. The very first thing that we need to do is write a Python function that will represent our mathematical function.

In [4]:
def profit(q):
    p = 300-0.5*q
    tr = p*q
    tc = 4000 + 45*q
    return tr - tc

This function will allow us to calculate profit at any output level based on our assumed total costs and demand curve. With this function, we can quickly calculate the gradient (in this case, just a simple derivative because our function is univariate) by calculating profit at two nearby points, and dividing by the horizontal distance between those points:

In [5]:
# Gradient at q=200

(profit(201) - profit(199))/2

55.0

    55.0



Thus, a one unit increase in output at $Q=200$ results in a $55 increase in profits. This is cool, but it isn't enough for us to find the point of maximum profit (the optimal point). For that, we will need to calculate LOTS of gradients in order to move along the function until we cannot increase profits any further.

Fortunately for us, `scipy` comes with optimization tools that will do all of the heavy lifting of the "search" for the optimal point. All that we need to do is frame our question algorithmically, and let `scipy` do the rest:

In [6]:
import numpy as np
from scipy.optimize import minimize

We start by importing the `minimize` function from `scipy.optimize`. Hang on! Weren't we working on a MAXIMIZATION problem?? What are we doing here?

Maximization and minimization are the **same thing**. To maximize a function, you can multiply that function by `-1` and then calculate the minimum of the new "upside-down" function. It is functionally equivalent. So, in computational optimization, we always minimize.

### Prepping for optimization

As we prepare to optimize, there are two common problems with our function that we may need to resolve:

1) When using `minimize` we can only pass an array of inputs, so we have to be careful to write our function accordingly
2) Our problem is concave, and so has a maximum
	- We need to restate it as a minimization problem

Problem 1 does not apply, since our function in univariate. In order to make our problem a minimization problem, we can flip our profit maximization function. We will simply return -1 * profit:

In [7]:
def profit(q):
    p = 300-0.5*q
    tr = p*q
    tc = 4000 + 45*q
    pi =  tr - tc # didn't name it profit because that is our function's name! Don't want to clutter our name space!
    return -1*pi

### Making the call to `minimize`

Now that our function is ready, it is time to minimize! The `minimize` function takes two arguments:
1. Our function that we want to optimize
2. A starting guess (as a vector)

Let's give it a try.

In [8]:
res = minimize(profit, [0]) # provide function and starting inputs
res

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -28512.499999980355
        x: [ 2.550e+02]
      nit: 3
      jac: [ 0.000e+00]
 hess_inv: [[ 1.000e+00]]
     nfev: 14
     njev: 7

          fun: -28512.499999980355
     hess_inv: array([[1.00000175]])
          jac: array([0.])
      message: 'Optimization terminated successfully.'
         nfev: 21
          nit: 3
         njev: 7
       status: 0
      success: True
            x: array([255.00019821])



That's it! No calculus, no searching, no checking gradients manually. `minimize` simply takes our function and our starting guess and brings us back the optimal choice. We get lots of information stored in the attributes of the `res` object:

- `fun` provides the value of the function (this is -1 times the profit level at the optimal output in our example)
- `hess_inv` and `jac` are measures of gradient and are used to determine how far to go and in which direction
- `message` should be self-explanatory
- `nfev` is the number of times the function (in this case `profit`) was evaluated during the search
- `nit` is the number of iterations it took to find the optimum
- `njev` is the number of times the Jacobian was estimated
- `status` is a code associated with the `message` and `success` atrributes
- `success` tells you whether or not an optimum was found (sometimes it cannot be easily found!)
- `x` probably the most important attribute. This tells us the optimal input value (in our case $Q$), or set of values depending on our function. In a regression context, this could represent the fitted coefficients!

Going forward, you will realize (especially because so many of them print the output as they optimize) just how many libraries in Python use this minimize function behind the scenes. It is used in `statsmodels`, `sklearn`, and many other high-profile libraries! Now that you know where it really happens (in `scipy`!), you'll be better able to troubleshoot the problems that will inevitably arise as you use statistical models.

**Solve-it!**

In this lesson we learned about optimization using SciPy. For the assignment this week, I would like you to build off of your `RegressionModel` class. You will add a Logistic Regression (Logit) method to your class, so that when the `regression_type` parameter is `logit`, Logistic Regression Results are returned.

Your job is to create the following functionality within your class object:
- a method (call it `logistic_regression`) that estimates the results of logistic regression using your `x` and `y` data frames, and using a likelihood function and gradient descent (DO NOT USE PREBUILT REGRESSION FUNCTIONS).
    - You need to write a function to calculate the Log-likelihood of your model
    - You need to implement gradient descent to find the optimal values of beta
    - You need to use your beta estimates, the model variance, and calculate the standard errors of the coefficients, as well as Z statistics and p-values
    - the results should be stored in a dictionary named `results`, where each variable name (including the intercept if `create_intercept` is `True`) is the key, and the value is another dictionary, with keys for `coefficient`, `standard_error`, `z_stat`, and `p_value`. The coefficient should be the log odds-ratio (which takes the place of the coefficients in OLS)
- a method called `fit_model` that uses the `self.regression_type` attribute to determine whether or not to run an OLS or Logistic Regression using the data provided. This method should call the correct regression method.
- an updated method (call it `summary`) that presents your regression results in a table
    - Columns should be: Variable name, Log odds-ratio value, standard error, z-statistic, and p-value, in that order.
    - Your summary table should have different column titles for OLS and Logistic Regressions! (think if statement...)

You only need to define the class. My code will create an instance of your class (be sure all the names match these instructions, and those from last week!), and provide data to run a regression. I will provide the same data to you, so that you can experiment and make sure that your code is functioning properly.

NOTE: I have created a [primer on Logistic regression](https://github.com/dustywhite7/Econ8320/blob/master/SlidesPDF/9-2%20-%20Logit%20Primer.pdf) to go with this assignment. See the Github slidesPDF folder

Put the code that you would like graded in the cell labeled `#si-exercise`. I recommend copying your code from the last assignment (in chapter 9 about linear regression and `numpy`), and continuing from there.



In [140]:
#si-exercise

import numpy as np
import scipy.stats as stat
import pandas as pd
from scipy.optimize import minimize

class RegressionModel(object):
    def __init__(self, x, y, create_intercept=True, regression_type='ols'):
        if isinstance(x, pd.DataFrame):
            self.x = x
        else:
            raise RuntimeError("Matrix 'x' is not a DataFrame.")
        if isinstance(y, pd.DataFrame) | isinstance(y, pd.Series):
            self.y = y
        else:
            raise RuntimeError("Matrix 'y' is not a DataFrame.")
        if isinstance(create_intercept, bool):
            self.create_intercept = create_intercept
            if self.create_intercept:
                self.add_intercept()
        else:
            raise RuntimeError("Parameter 'create_intercept' must be a boolean value.")
        if isinstance(regression_type, str):
            if regression_type=="ols":
                self.regression_type=regression_type
            elif regression_type=='logit':
                self.regression_type=regression_type
            else:
                raise RuntimeError("Only OLS and Logistic regressions ('ols' or 'logit', respectively) are supported")
        else:
            raise RuntimeError("Parameter 'regression_type' must be a string with value 'ols' or 'logit'.")

    def add_intercept(self):
        self.x = self.x.assign(intercept=pd.Series([1]*np.shape(self.x)[0]))

    def ols_regression(self):
        x = self.x
        y = self.y
        n, k = np.shape(x)
        beta = np.dot(np.linalg.solve(x.T.dot(x), np.eye(k)), x.T.dot(y))
        eps = y - x.dot(beta)
        shat = eps.T.dot(eps)/(n-k)
        covar = shat * np.linalg.solve(x.T.dot(x), np.eye(k))
        var = np.diag(covar)
        serror = np.asarray([np.sqrt(i) for i in var])
        tstat = np.asarray([i[1]/serror[i[0]] for i in enumerate(beta)])
        pval = stat.t.sf(tstat, n-k)

        self.results = dict()

        for i in enumerate(self.x.columns):
            self.results[i[1]] = {
                    'coefficient' : beta[i[0]],
                    'standard_error' : serror[i[0]],
                    't_stat' : tstat[i[0]],
                    'p_value' : pval[i[0]]
                    }

    def logistic_regression(self):
      x = self.x
      y = self.y
      n, k = np.shape(x)
      #beta random values
      beta = np.random.randn(k)
      logtransform = x.dot(beta)
      p = np.exp(logtransform) / (1 + np.exp(logtransform))

      def logistic_function(x, beta):
        return 1 / (1 + np.exp(-x.dot(beta)))

      def log_likelihood(beta):
        tll = 0

        for i in range(len(y)):
          xi = x.iloc[i]
          yi = y.iloc[i]

          linear_comb = np.dot(xi, beta)

          p = np.exp(linear_comb) / (1 + np.exp(linear_comb))

          oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)

          tll += oll

        return -tll

      def gradient(beta):
        linear_comb = np.dot(x, beta)

        p = np.exp(linear_comb) / (1 + np.exp(linear_comb))
        print(-np.dot(x.T, y - p))
        return -np.dot(x.T, y - p)

      minim = minimize(log_likelihood, beta, jac=gradient)
      beta = minim.x
      print(beta)

      p = logistic_function(x, beta)

      sigmahat2 = n * np.mean(p * (1 - p))
      covmatrix = sigmahat2 * np.linalg.inv(x.T @ x)

      se = np.sqrt(np.diag(covmatrix))
      zstat = beta / se
      pvals = [2 * (1 - stat.norm.cdf(np.abs(i))) for i in zstat]

      self.results = {}
      for i in enumerate(self.x.columns):
        self.results[i[1]] = {
            'coefficient': beta[i[0]],
            'standard_error': se[i[0]],
            'z_stat': zstat[i[0]],
            'p_value': pvals[i[0]]
        }

    def fit_model(self):
        if self.regression_type == 'ols':
            self.ols_regression()
        elif self.regression_type == 'logit':
            self.logistic_regression()
        else:
            raise RuntimeError("Wrong Regression Input")

    def summary(self):
      if self.regression_type == 'ols':
        print("Variable Name".ljust(25) + "| Coefficient".ljust(15) + "| Standard Error".ljust(17) + "| T-Statistic".ljust(15) + "| P-Value".ljust(15) + "\n" + "-"*85)
        for i in self.results:
            print("{}".format(i).ljust(25) + "| {}".format(round(self.results[i]['coefficient'], 3)).ljust(15) + "| {}".format(round(self.results[i]['standard_error'], 3)).ljust(17) + "| {}".format(round(self.results[i]['t_stat'], 3)).ljust(15) + "| {}".format(round(self.results[i]['p_value'], 3)).ljust(15))
      elif self.regression_type == 'logit':
        print("Variable Name".ljust(25) + "| Log odds-ratio".ljust(15) + "| Standard Error".ljust(17) + "| z-statistic".ljust(15) + "| p-value".ljust(15) + "\n" + "-"*85)
        for i in self.results:
              print("{}".format(i).ljust(25) + "| {}".format(round(self.results[i]['coefficient'], 3)).ljust(15) + "| {}".format(round(self.results[i]['standard_error'], 3)).ljust(17) + "| {}".format(round(self.results[i]['z_stat'], 3)).ljust(15) + "| {}".format(round(self.results[i]['p_value'], 3)).ljust(15))
      else:
        raise RuntimeError("Nice try sport, no Probit here :)")


In [141]:
data = pd.read_csv("/content/sample_data/assignment8Data.csv")
x = data.loc[:100, ['sex','age','educ']]
y = data.loc[:100, 'white']
reg = RegressionModel(x, y, create_intercept=True, regression_type='logit')

In [142]:
reg.fit_model()

<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: divide by zero encountered in log
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: invalid value encountered in scalar multiply
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: divide by zero encountered in log
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: invalid value encountered in scalar multiply
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: divide by zero encountered in log
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: invalid value encountered in scalar multiply
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: divide by zero encountered in log
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython

[  7. 209.  32.   4.]
[ -144. -4866.  -745.   -97.]
[ -144. -4866.  -745.   -97.]
[  7. 209.  32.   4.]
[  7. 209.  32.   4.]
[ -144. -4866.  -745.   -97.]
[  7.         208.99999999  32.           4.        ]
[  6.99999999 208.99999985  31.99999996   3.99999999]
[ -143.99999865 -4865.99997788  -744.99999132   -96.99999883]
[  6.99999996 208.99999935  31.99999982   3.99999996]
[  6.99999984 208.99999727  31.99999923   3.99999985]
[ -122.73565638 -4496.38438186  -617.90951834   -82.89994166]
[  6.99999967 208.99999424  31.99999836   3.99999969]
[  6.99999933 208.9999882   31.99999665   3.99999936]
[  6.32020868 161.0925641   30.47868775   3.42663307]
[  -90.23081966 -3688.78740357  -431.65785073   -61.74526319]
[  5.50688298 109.98628108  28.54074523   2.83125354]
[  3.36437647 -10.71855434  22.80965766   1.44467205]
[ 3.52243173 -2.12726902 23.25966259  1.54411421]
[ 3.49365734 -3.6949604  23.17802678  1.52598682]
[ 3.4936565  -3.69500632 23.17802439  1.52598628]
[ 3.49365734 -3.694960

<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: divide by zero encountered in log
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: invalid value encountered in scalar multiply
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: divide by zero encountered in log
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: invalid value encountered in scalar multiply
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: divide by zero encountered in log
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: invalid value encountered in scalar multiply
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython-input-140-eb4eab5e2158>:83: RuntimeWarning: divide by zero encountered in log
  oll = yi * np.log(p) + (1 - yi) * np.log(1 - p)
<ipython

[ -106.44839815 -4141.16482339  -526.81447394   -72.40561455]
[  6.9999999  208.99999828  31.99999951   3.99999991]
[  6.99885417 208.94684011  31.99614781   3.99905144]
[ 4.76088243 66.74216354 26.64068677  2.3350168 ]
[ -144. -4866.  -745.   -97.]
[ 3.35053741 -1.51749694 22.42633738  1.48128156]
[ 2.16656782 -3.32131278 16.33566166  0.89453294]
[ -105.72535855 -3119.89101902  -575.21620253   -69.87361059]
[  0.90949841 -29.37415481  10.40612142   0.13363631]
[ -1.34638715 -42.42108532  -5.20372333  -1.39271709]
[ 8.19530085e-01  2.75524057e+01  3.51144995e+00 -4.36969280e-03]
[ 0.27325232 10.89116613 -1.47806686 -0.33997701]
[ 0.13466705  8.22712538 -1.48608544 -0.15270283]
[ 0.04843163  2.04248659 -0.11253833 -0.01326066]
[-0.05748538 -2.01711125 -0.29079461 -0.03780351]
[-0.00204977 -0.16418217  0.01004245 -0.00219919]
[-0.00135789 -0.04869682 -0.0012337  -0.00076549]
[-1.47759587e-04 -1.50903213e-03 -4.51477050e-04 -5.94425223e-05]
[-2.06453999e-05 -6.12232097e-05 -7.09058681e-05

In [143]:
reg.results

{'sex': {'coefficient': -1.1229160127656386,
  'standard_error': 0.39567173057049615,
  'z_stat': -2.8379990937097554,
  'p_value': 0.004539731153691884},
 'age': {'coefficient': -0.007012504023604423,
  'standard_error': 0.010772765271913197,
  'z_stat': -0.6509474444678987,
  'p_value': 0.51508041335937},
 'educ': {'coefficient': -0.04648577002346137,
  'standard_error': 0.10041501867508354,
  'z_stat': -0.462936427606283,
  'p_value': 0.6434099356716487},
 'intercept': {'coefficient': 5.735437548951124,
  'standard_error': 1.1200645945357108,
  'z_stat': 5.120631057290564,
  'p_value': 3.0451489863025927e-07}}